# Create table

***data dimension***

In [101]:
import pandas as pd

# Load your dataset
data = pd.read_csv('final_result.csv')

# Extract unique dates and corresponding month and year
date_dimension = data[['date']].copy()
date_dimension = date_dimension.drop_duplicates()
date_dimension['dateID'] = range(1, len(date_dimension) + 1)
date_dimension['date'] = pd.to_datetime(date_dimension['date'])
date_dimension['month'] = date_dimension['date'].dt.strftime('%B')
date_dimension['year'] = date_dimension['date'].dt.year

# Select required columns for the data dimension
date_dimension = date_dimension[['dateID', 'date', 'month', 'year']]

# Save the date dimension to a new CSV file
date_dimension.to_csv('date_dimension.csv', index=False)


***province dimension***

In [102]:
import pandas as pd

# Load your dataset
data = pd.read_csv('final_result.csv')

# Extract unique provinces
province_dimension = data[['province', 'prname']].drop_duplicates()


# Save the province dimension to a new CSV file
province_dimension.to_csv('province_dimension.csv', index=False)


***Vaccination dimension***

In [103]:
import pandas as pd

# Load your dataset
data = pd.read_csv('final_result.csv')

# Extract necessary columns for the Vaccination Dimension
vaccination_dimension = data[['date', 'province', 'partial', 'fully', 'booster']].copy()

# Load Province Dimension for Foreign Key
province_dimension = pd.read_csv('province_dimension.csv')

# Merge Province Dimension to get ProvinceID
vaccination_dimension = vaccination_dimension.merge(province_dimension, how='left', left_on='province', right_on='province')


date_dimension = pd.read_csv('date_dimension.csv')
# Merge Province Dimension to get dateID
vaccination_dimension = vaccination_dimension.merge(date_dimension, how='left', left_on='date', right_on='date')

# Drop redundant columns
vaccination_dimension.drop(['prname','date'], axis=1, inplace=True)

vaccination_dimension['vacID'] = range(1, len(vaccination_dimension) + 1)

vaccination_dimension = vaccination_dimension[['vacID','dateID', 'province', 'partial', 'fully', 'booster']].drop_duplicates()

# Save the Vaccination Dimension to a new CSV file
vaccination_dimension.to_csv('vaccination_dimension.csv', index=False)


***Covid19 Metric dimension***

In [104]:
import pandas as pd

# Load your dataset
data = pd.read_csv('final_result.csv')

# Extract necessary columns for the COVID19 Metrics Dimension
covid_dimension = data[['date', 'province', 'totalcases', 'numdeaths', 'recovery']].copy()

# Load Date Dimension for Foreign Key
date_dimension = pd.read_csv('date_dimension.csv')

# Merge Date Dimension to get DateID
covid_dimension = covid_dimension.merge(date_dimension, how='left', left_on='date', right_on='date')

# # Drop redundant columns
# covid_dimension.drop(['date'], axis=1, inplace=True)

covid_dimension['covidID'] = range(1, len(covid_dimension) + 1)

covid_dimension = covid_dimension[['covidID', 'dateID','province', 'totalcases', 'numdeaths', 'recovery']].drop_duplicates()

# Save the COVID19 Metrics Dimension to a new CSV file
covid_dimension.to_csv('covid19_metrics_dimension.csv', index=False)


# SQL create

date dimension

In [105]:
import csv

# Define your CSV file path
csv_file_path = 'date_dimension.csv'

date_create_table_statement = """
CREATE TABLE DateDimension (
    dateID INTEGER PRIMARY KEY,
    date DATE,
    month VARCHAR(20),
    year INTEGER
);
"""

# Read data from the CSV file and store it in a list of tuples
data_list = []
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        # Assuming each row in the CSV corresponds to one tuple of data
        data_list.append(tuple(row))

# Generate SQL insert statements
date_insert_statements = []
count = 0
for row in data_list:
    if count == 0:
        count = 1
        continue
    values = ', '.join([f"'{str(value)}'" if isinstance(value, str) else str(value) for value in row])
    date_insert_statements.append(f"INSERT INTO DateDimension VALUES ({values});")

province dimension

In [106]:
import csv

# Define your CSV file path
csv_file_path = 'province_dimension.csv'

pro_create_table_statement = """
CREATE TABLE ProvinceDimension (
    province VARCHAR(2) PRIMARY KEY,
    prname VARCHAR(50)
);
"""

# Read data from the CSV file and store it in a list of tuples
data_list = []
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        # Assuming each row in the CSV corresponds to one tuple of data
        data_list.append(tuple(row))

# Generate SQL insert statements
pro_insert_statements = []
count = 0
for row in data_list:
    if count == 0:
        count = 1
        continue
    values = ', '.join([f"'{str(value)}'" if isinstance(value, str) else str(value) for value in row])
    pro_insert_statements.append(f"INSERT INTO ProvinceDimension VALUES ({values});")


vaccination dimension

In [107]:
import csv

# Define your CSV file path
csv_file_path = 'vaccination_dimension.csv'

vac_create_table_statement = """
CREATE TABLE VaccinationDimension (
    vacID INTEGER PRIMARY KEY,
    dateID INTEGER,
    province VARCHAR(50),
    partial INTEGER,
    fully INTEGER,
    booster INTEGER,
    FOREIGN KEY (dateID) REFERENCES DateDimension(dateID),
    FOREIGN KEY (province) REFERENCES ProvinceDimension(province)
);
"""
vac_update_statement = """
-- Add new columns for change in partial, fully, and booster
ALTER TABLE VaccinationDimension
ADD COLUMN partial_change INT,
ADD COLUMN fully_change INT,
ADD COLUMN booster_change INT;

-- Set the changes to 0 for the first date
UPDATE VaccinationDimension
SET partial_change = 0, fully_change = 0, booster_change = 0
WHERE dateID = (SELECT MIN(dateID) FROM VaccinationDimension);

-- Calculate the changes for subsequent dates
WITH cte AS (
  SELECT
    vacID,
    dateID,
    province,
    partial,
    fully,
    booster,
    LAG(partial, 1) OVER (PARTITION BY province ORDER BY dateID) AS prev_partial,
    LAG(fully, 1) OVER (PARTITION BY province ORDER BY dateID) AS prev_fully,
    LAG(booster, 1) OVER (PARTITION BY province ORDER BY dateID) AS prev_booster
  FROM VaccinationDimension
)
UPDATE VaccinationDimension vd
SET partial_change = vd.partial - cte.prev_partial,
    fully_change = vd.fully - cte.prev_fully,
    booster_change = vd.booster - cte.prev_booster
FROM cte
WHERE vd.vacID = cte.vacID
  AND vd.dateID = cte.dateID
  AND vd.province = cte.province
  AND vd.dateID > (SELECT MIN(dateID) FROM VaccinationDimension);
"""

# Read data from the CSV file and store it in a list of tuples
data_list = []
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        # Assuming each row in the CSV corresponds to one tuple of data
        data_list.append(tuple(row))

# Generate SQL insert statements
vac_insert_statements = []
count = 0
for row in data_list:
    if count == 0:
        count = 1
        continue
    values = ', '.join([f"'{str(value)}'" if isinstance(value, str) else str(value) for value in row])
    vac_insert_statements.append(f"INSERT INTO VaccinationDimension VALUES ({values});")


Covid Metric Dimension

In [108]:
import csv

# Define your CSV file path
csv_file_path = 'covid19_metrics_dimension.csv'

cov_create_table_statement = """
CREATE TABLE COVID19MetricsDimension (
    covidID INTEGER PRIMARY KEY,
    dateID INTEGER,
    province VARCHAR(50),
    totalCases INTEGER,
    numDeaths INTEGER,
    recovery INTEGER,
    FOREIGN KEY (dateID) REFERENCES DateDimension(dateID),
    FOREIGN KEY (province) REFERENCES ProvinceDimension(province)
);
"""

# Read data from the CSV file and store it in a list of tuples
data_list = []
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        # Assuming each row in the CSV corresponds to one tuple of data
        data_list.append(tuple(row))

# Generate SQL insert statements
cov_insert_statements = []
count = 0
for row in data_list:
    if count == 0:
        count = 1
        continue
    values = ', '.join([f"'{str(value)}'" if isinstance(value, str) else str(value) for value in row])
    cov_insert_statements.append(f"INSERT INTO COVID19MetricsDimension VALUES ({values});")


In [109]:
create_table_statement = """
CREATE TABLE FactTable (
    dateID INTEGER,
    vacID INTEGER,
    covidID INTEGER,
    province VARCHAR(2),
    totalCases INTEGER,
    recovery INTEGER,
    death INTEGER,
    PRIMARY KEY (dateID, vacID, covidID, province),
    FOREIGN KEY (dateID) REFERENCES DateDimension(dateID),
    FOREIGN KEY (vacID) REFERENCES VaccinationDimension(vacID),
    FOREIGN KEY (covidID) REFERENCES COVID19MetricsDimension(covidID),
    FOREIGN KEY (province) REFERENCES ProvinceDimension(province)
);
"""

insert_statements = """
INSERT INTO FactTable (DateID, VacID, CovidID, Province, totalCases,recovery, Death)
SELECT
    dd.DateID,
    vd.VacID,
    cmd.CovidID,
    pd.Province,
    cmd.totalCases AS TotalCases,
    cmd.Recovery AS Recovery,
    cmd.NumDeaths AS Death
FROM
    DateDimension dd
    JOIN VaccinationDimension vd ON dd.DateID = vd.DateID
    JOIN COVID19MetricsDimension cmd ON dd.DateID = cmd.DateID AND vd.Province = cmd.Province
    JOIN ProvinceDimension pd ON vd.Province = pd.Province;"""

update_statement = """
-- Add new columns for change in totalCases, recovery, and death
ALTER TABLE FactTable
ADD COLUMN totalCases_change INT,
ADD COLUMN recovery_change INT,
ADD COLUMN death_change INT;

-- Set the changes to 0 for the first date
UPDATE FactTable
SET totalCases_change = 0, recovery_change = 0, death_change = 0
WHERE dateID = (SELECT MIN(dateID) FROM FactTable);

-- Calculate the changes for subsequent dates
WITH cte AS (
  SELECT
    dateID,
    vacID,
    covidID,
    province,
    totalCases,
    recovery,
    death,
    LAG(totalCases, 1) OVER (PARTITION BY province ORDER BY dateID) AS prev_totalCases,
    LAG(recovery, 1) OVER (PARTITION BY province ORDER BY dateID) AS prev_recovery,
    LAG(death, 1) OVER (PARTITION BY province ORDER BY dateID) AS prev_death
  FROM FactTable
)
UPDATE FactTable ft
SET totalCases_change = ft.totalCases - cte.prev_totalCases,
    recovery_change = ft.recovery - cte.prev_recovery,
    death_change = ft.death - cte.prev_death
FROM cte
WHERE ft.dateID = cte.dateID
  AND ft.vacID = cte.vacID
  AND ft.covidID = cte.covidID
  AND ft.province = cte.province
  AND ft.dateID > (SELECT MIN(dateID) FROM FactTable);

"""

In [110]:
with open('dimension_creating.sql', 'w') as f:

    f.write('\n')
    f.write(date_create_table_statement)
    f.write('\n')
    f.write(pro_create_table_statement)
    f.write('\n')
    f.write(cov_create_table_statement)
    f.write('\n')
    f.write(vac_create_table_statement)
    f.write('\n')
    f.write(create_table_statement)

    for data in date_insert_statements:
        f.write(data + '\n')
    for data in pro_insert_statements:
        f.write(data + '\n')
    for data in cov_insert_statements:
        f.write(data + '\n')
    for data in vac_insert_statements:
        f.write(data + '\n')
    f.write(vac_update_statement + '\n')
    f.write(insert_statements + '\n')
    f.write(update_statement + '\n')